# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.<br>
Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. <br>
Эти данные помогут сформулировать ценностное предложение для нового продукта.<br>


### Цели исследования

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Подключение к базе данных

In [1]:
# Импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 

# Устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'], 
                                                         db_config['db'])

# Сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Знакомство с таблицами

In [2]:
# Формируем запрос к базе SQL
# Воспользуемся функцией для просмотра первых строк сразу всех пяти таблиц

for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    query = ''' SELECT * 
                FROM {}; 
    '''.format(table) 

    request = pd.io.sql.read_sql(query, con = engine) 
    
    print()
    print(f'Таблица {table}')
    display(request.head(3))
    print('*'*100)


Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


****************************************************************************************************

Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


****************************************************************************************************

Таблица ratings


...

****************************************************************************************************

Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


****************************************************************************************************

Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


****************************************************************************************************


### Задачи проекта

#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [3]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'::date

"""
result = pd.io.sql.read_sql(query, con = engine)
result

,count
0,819


Ответ: после 1 января 2000 года вышло 819 книг.

#### Посчитаем количество обзоров и среднюю оценку для каждой книги

In [4]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS review_count,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r ON b.book_id = r.book_id
     LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, 
         review_count DESC

"""
result = pd.io.sql.read_sql(query, con = engine)
result

,title,review_count,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Мы получили таблицу - список книг, отсортированных по убыванию количества обзоров и среднего рейтинга.

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц
чтобы исключить из анализа брошюры

In [5]:
query = """
SELECT publishers.publisher AS name_publisher,
       COUNT(books.book_id) AS books_count
FROM publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY name_publisher
ORDER BY books_count DESC
LIMIT 5

"""
result = pd.io.sql.read_sql(query, con = engine)
result

,name_publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Мы получили таблицу - список издательств, отсортированных по убыванию количеством книг, состоящих из более чем 50 страниц.<br>
Наибольшее количество таких книг выпустило издательство Penguin Books.

#### Определим автора с самой высокой средней оценкой книг 
учитываем только книги с 50 и более оценками

In [6]:
query = """
SELECT author,
       AVG(rating) AS mean_rating
FROM authors AS a
     JOIN books AS b ON a.author_id = b.author_id
     JOIN ratings AS r ON b.book_id = r.book_id
WHERE b.book_id IN (SELECT book_id
                    FROM ratings 
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50
                    ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY mean_rating DESC
LIMIT 5

"""
result = pd.io.sql.read_sql(query, con = engine)
result

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Автор с самой высокой средней оценкой книг(среди книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré.

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [7]:
query = """
SELECT AVG(count.reviews_avg)
FROM (SELECT username,
             COUNT(review_id) AS reviews_avg
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 50)
      GROUP BY username) 
      AS count

"""
result = pd.io.sql.read_sql(query, con = engine)
result

,avg
0,24.333333


В среднем получилось 24 обзора пользователей, которые поставили более 50 оценок.